# Linear Regression


$$\begin{align*}
\hat{y} = \theta_0 + \theta_1x_1 + \theta_2x_2 + ... + \theta_nx_n
\end{align*}$$

A weighted sum of the input features, plus a constant called the bias term (also called the intercept term)

Vectorized form
$$
\begin{align*}
\hat{y} = h_\theta(\mathbf{x}) = \mathbf{\theta} \cdot \mathbf{x}
\end{align*}
$$

- $\mathbf{\theta}$: model's parameter vector, containing the bias term $\theta_0$ and feature weights $\theta_1$ to $\theta_n$
- $\mathbf{x}$ is the instance's feature vector. containing $x_0$ to $x_n$, with $x_0$ always equal to 1.
- $\mathbf{\theta} \cdot \mathbf{x}$ is the dot product of the vectors $\theta$ and $x$, which is of course equal to $\theta_0x_0+\theta_1x_1+\theta_2x_2 \dots +\theta_nx_n $
- $h_0$: hypothesis function.

In order to train a Linear Regression model, we need a measure of how well the model fits the training data. In practice, it is simpler to minimiye the Mean Square Error (MSE) than the Root Mean Square Error (RMSE).

$$
\begin{align*}
MSE(\mathbf{X}, h_\theta) = \frac{1}{m}\sum_{i=1}^{m}\left(\mathbf{\theta^T}\mathbf{x}^{(i)}-y^{(i)}\right)
\end{align*}
$$
